# Inspect TDM File

This notebook allows you to:
- Open a `.tdm` file and its corresponding `.tdx` file.
- Explore the metadata and time histories of the experimental data.
- Visualize specific channels interactively.

In [ ]:
import os
import pandas as pd
import tdm_loader as tdm
import numpy as np
import matplotlib.pyplot as plt
from tkinter import Tk, filedialog

In [ ]:
def select_tdm_file():
    """
    Opens a dialog to select a TDM file.
    
    Returns:
        str: Path to the selected TDM file.
    """
    root = Tk()
    root.withdraw()
    root.attributes('-topmost', True)
    file_path = filedialog.askopenfilename(
        title="Select TDM File",
        filetypes=[("TDM Files", "*.tdm;*.TDM")]
    )
    root.destroy()
    return file_path

In [ ]:
def load_tdm_data(tdm_file, fs=1000):
    """
    Load data from a TDM file.
    
    Args:
        tdm_file (str): Path to the TDM file.
        fs (int): Sampling frequency in Hz (default: 1000 Hz).
    
    Returns:
        tuple: (metadata, DataFrame of channel data).
    """
    try:
        data = tdm.OpenFile(tdm_file)
        d = {}
        group_index = 0
        channel_index = 0
        while True:
            try:
                channel_name = data.channel_name(group_index, channel_index)
                channel_data = data.channel(group_index, channel_index)
                d[channel_name] = channel_data
                channel_index += 1
            except IndexError:
                break

        df = pd.DataFrame.from_dict(d)
        time = np.arange(len(df)) / fs
        df['time'] = time
        df = df.set_index('time')
        
        metadata = {
            "Number of Channels": channel_index,
            "Channel Length": len(df),
            "File Path": tdm_file
        }

        return metadata, df
    except Exception as e:
        print(f"Error loading TDM file: {e}")
        return None, None

In [ ]:
# Manually specify the file path or use the file dialog
# tdm_file = "path_to_your_file.tdm"
tdm_file = select_tdm_file()

if not tdm_file:
    print("No file selected.")
else:
    metadata, df = load_tdm_data(tdm_file)
    if metadata and df is not None:
        print("Metadata:")
        for key, value in metadata.items():
            print(f"{key}: {value}")

        print("\nChannel Names:")
        print(df.columns.values)

        channel_to_plot = df.columns[0]
        plt.figure(figsize=(12, 6))
        plt.plot(df[channel_to_plot], label=channel_to_plot)
        plt.title(f"Channel: {channel_to_plot}")
        plt.xlabel("Time [s]")
        plt.ylabel("Engineering Value")
        plt.legend()
        plt.grid()
        plt.show()